In [219]:
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")
import xgboost as xgb
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report,r2_score,mean_squared_error

In [220]:
df = pd.read_csv('flights_final.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97682 entries, 0 to 97681
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mkt_carrier          97682 non-null  int64  
 1   origin               97682 non-null  int64  
 2   dest                 97682 non-null  int64  
 3   dep_delay            97682 non-null  float64
 4   arr_delay            97682 non-null  float64
 5   crs_elapsed_time     97682 non-null  int64  
 6   actual_elapsed_time  97682 non-null  float64
 7   air_time             97682 non-null  float64
 8   distance             97682 non-null  int64  
 9   month                97682 non-null  int64  
 10  weekday              97682 non-null  int64  
 11  dep_hour             97682 non-null  int64  
 12  time_cat             97682 non-null  int64  
 13  avg_delay            97682 non-null  float64
dtypes: float64(5), int64(9)
memory usage: 10.4 MB


In [221]:
df1 = pd.read_csv('test_df.csv')

In [222]:
df1 = df1.drop(['Unnamed: 0'],axis=1)


In [223]:
df1 = df1.rename(columns={'dep_time_format':'dep_hour'})

In [224]:
df1.head()

,mkt_carrier,origin,dest,crs_elapsed_time,distance,dep_hour,weekday,month
0,9,246,312,95,363,18,2,1
1,9,246,312,90,363,11,2,1
2,9,246,318,70,333,20,2,1
3,9,246,318,75,333,13,2,1
4,9,246,318,80,333,9,2,1


In [225]:
df1.values

array([[  9, 246, 312, ...,  18,   2,   1],
       [  9, 246, 312, ...,  11,   2,   1],
       [  9, 246, 318, ...,  20,   2,   1],
       ...,
       [  3,  20, 358, ...,  17,   1,   1],
       [  3, 358,  20, ...,  18,   1,   1],
       [  3, 283, 196, ...,   6,   1,   1]])

In [226]:
df.head()

,mkt_carrier,origin,dest,dep_delay,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance,month,weekday,dep_hour,time_cat,avg_delay
0,2,74,145,-6.0,-4.0,59,61.0,31.0,83,11,2,16,2,0.363720
1,2,249,95,40.0,24.0,70,54.0,34.0,175,12,3,17,3,50.452426
2,7,349,316,-6.0,0.0,120,126.0,100.0,727,10,3,9,2,0.363720
3,3,22,363,19.0,31.0,71,83.0,43.0,250,6,0,14,3,50.452426
4,6,118,127,19.0,-10.0,186,157.0,137.0,1065,11,2,17,1,-10.006351


In [227]:
X = df.drop(['time_cat','arr_delay','dep_delay','actual_elapsed_time','avg_delay','air_time'],axis=1)
y = df['time_cat']

In [228]:
y.value_counts()

0    25313
1    25034
2    23782
3    23553
Name: time_cat, dtype: int64

In [229]:
# Standard Scaler creation using the fit_transform() method
scaler = StandardScaler()
scaled_df = scaler.fit_transform(X)

In [235]:
xg_reg = xgb.XGBClassifier(n_estimators=100)
#clf = GridSearchCV(xg_reg,params)

In [236]:
# Perform the dataset split
X_test, X_train, y_test, y_train = train_test_split(scaled_df, y, test_size=0.30, random_state=42)

In [250]:
xg_reg.fit(X_train, y_train)

training_preds = xg_reg.predict(X_train)
y_pred = xg_reg.predict(df1.values)

[13:07:51] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [251]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [241]:
# training_accuracy = accuracy_score(y_train, y_pred)
val_accuracy = accuracy_score(y_test, y_pred)

# print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Validation accuracy: 34.14%


In [242]:
confusion_matrix(y_test, y_pred)

array([[8034, 3712, 2484, 3544],
       [5101, 5220, 3618, 3637],
       [4608, 4474, 3659, 3916],
       [4246, 2969, 2721, 6434]])

In [243]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.37      0.45      0.40     17774
           1       0.32      0.30      0.31     17576
           2       0.29      0.22      0.25     16657
           3       0.37      0.39      0.38     16370

    accuracy                           0.34     68377
   macro avg       0.34      0.34      0.34     68377
weighted avg       0.34      0.34      0.34     68377



In [244]:
y_pred

array([3, 2, 1, ..., 0, 0, 3])

In [ ]:
pred.nunique()

In [ ]:
pred = pd.DataFrame(y_pred)

### Hyperparameter tuning 

In [ ]:
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : 3000,
        'objective': ['reg:squarederror']}

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [ ]:
hyperParameterTuning(X_train,y_train)

In [ ]:
xgb_model = XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = 0.1,
        learning_rate = 0.0001,
        max_depth = 4,
        min_child_weight = 1,
        n_estimators = 3000,
        alpha = 5)

xgb_model.fit(X_train, y_train)
training_preds = xgb_model.predict(X_train)
y_pred = xgb_model.predict(X_test)



In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %.4f" % (rmse))

In [ ]:
xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [300,300]
plt.show()

### Regressor

In [ ]:
df.head()

In [ ]:
X = df.drop(['time_cat','arr_delay','dep_delay','actual_elapsed_time','air_time'],axis=1)
y = df['arr_delay']

In [ ]:
X

In [ ]:
X_test, X_train, y_test, y_train = train_test_split(scaled_df, y, test_size=0.30, random_state=42)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 1.0, learning_rate = 0.001, gamma = 2.5, min_child_weight =3,
                max_depth = 5, alpha =20, n_estimators = 8000)

xg_reg.fit(X_train,y_train)

In [ ]:
y_pred = xg_reg.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
r2 = r2_score(y_test,y_pred)

print('rmse: ',rmse)
print('r2_score:', r2)